# Keras Tuner- Decide Number of Hidden Layers And Neuron In Neural Network

In [ ]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [ ]:
df=pd.read_csv('../input/air-quality/Real_Combine.csv')
df.head()

In [ ]:
X=df.iloc[:,:-1] ## independent features
y=df.iloc[:,-1] ## dependent features

### Hyperparameters
1. How many number of hidden layers we should have?
1. How many number of neurons we should have in hidden layers?
1. Learning Rate

In [ ]:

def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,  # min value for neuron
                                            max_value=512,  # max value
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3, # 5*3 =15 trails(iteration) for each instances
    directory='project',
    project_name='Air Quality Index')

In [ ]:
tuner.search_space_summary()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

In [ ]:
tuner.results_summary()